# 2. Price Prediction Model Training (XGBoost)

Notebook này train model XGBoost để dự đoán giá thuê phòng/nhà dựa trên các features.

## Quy trình:
1. Load dữ liệu đã chuẩn bị từ notebook 1
2. Feature engineering và preprocessing
3. Train XGBoost model với hyperparameter tuning
4. Đánh giá model (MAE, RMSE, R²)
5. Lưu model vào file .pkl
6. Test với dữ liệu mẫu

## 1. Cài đặt thư viện

In [ ]:
!pip install xgboost scikit-learn pandas numpy matplotlib seaborn joblib

## 2. Import libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb
import joblib
from google.colab import files

# Cấu hình hiển thị
pd.set_option('display.max_columns', None)
plt.style.use('ggplot')
sns.set_palette('husl')

## 3. Load dữ liệu

Upload file `training_data.csv` từ notebook 1 hoặc từ máy của bạn.

In [ ]:
# Upload file CSV
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# Load data
df = pd.read_csv(filename)
print(f"Đã load {len(df)} records")
print(f"\nCác cột: {df.columns.tolist()}")
df.head()

## 4. Exploratory Data Analysis (EDA)

In [ ]:
# Thống kê cơ bản
print("=== Thống kê giá thuê ===")
print(df['price'].describe())
print(f"\nGiá trung bình: {df['price'].mean():,.0f} VNĐ")
print(f"Giá trung vị: {df['price'].median():,.0f} VNĐ")

# Kiểm tra missing values
print("\n=== Missing Values ===")
print(df.isnull().sum())

# Phân bố giá
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histogram
axes[0].hist(df['price'], bins=50, edgecolor='black')
axes[0].set_xlabel('Giá thuê (VNĐ)')
axes[0].set_ylabel('Số lượng')
axes[0].set_title('Phân bố giá thuê')

# Boxplot
axes[1].boxplot(df['price'])
axes[1].set_ylabel('Giá thuê (VNĐ)')
axes[1].set_title('Boxplot giá thuê')

plt.tight_layout()
plt.show()

In [ ]:
# Correlation heatmap
plt.figure(figsize=(12, 10))
correlation_matrix = df.corr()
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            square=True, linewidths=0.5)
plt.title('Ma trận tương quan giữa các features')
plt.tight_layout()
plt.show()

# Top features có correlation cao với price
print("\n=== Top 10 features tương quan với giá ===")
price_corr = correlation_matrix['price'].sort_values(ascending=False)
print(price_corr.head(10))

## 5. Feature Engineering

In [ ]:
# Tạo features mới
df['price_per_sqm'] = df['price'] / df['area']
df['total_amenities'] = (df['has_wifi'] + df['has_parking'] + 
                          df['has_air_conditioner'] + df['has_water_heater'] +
                          df['has_kitchen'] + df['has_fridge'] + 
                          df['has_washing_machine'] + df['has_tv'] +
                          df['has_bed'] + df['has_wardrobe'])

df['amenity_score'] = df['total_amenities'] / 10.0  # Chuẩn hóa 0-1

# Interaction features
df['area_rooms'] = df['area'] * df['bedrooms']
df['area_amenities'] = df['area'] * df['amenity_score']

print("Đã tạo features mới:")
print("- price_per_sqm: Giá trên mét vuông")
print("- total_amenities: Tổng số tiện nghi")
print("- amenity_score: Điểm tiện nghi (0-1)")
print("- area_rooms: Tương tác diện tích x phòng ngủ")
print("- area_amenities: Tương tác diện tích x tiện nghi")

## 6. Chuẩn bị dữ liệu train/test

In [ ]:
# Chọn features để train
feature_columns = [
    # Location
    'province_encoded', 'district_encoded',
    # Property
    'area', 'bedrooms', 'bathrooms', 'floor',
    # Amenities
    'has_wifi', 'has_parking', 'has_air_conditioner', 'has_water_heater',
    'has_kitchen', 'has_fridge', 'has_washing_machine', 'has_tv',
    'has_bed', 'has_wardrobe',
    # Engineered features
    'total_amenities', 'amenity_score', 'area_rooms', 'area_amenities'
]

X = df[feature_columns]
y = df['price']

# Train/test split (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Training set: {len(X_train)} samples")
print(f"Test set: {len(X_test)} samples")
print(f"\nFeatures: {len(feature_columns)}")
print(feature_columns)

## 7. Train XGBoost Model với Hyperparameter Tuning

In [ ]:
# Baseline model
print("=== Training Baseline XGBoost Model ===")
baseline_model = xgb.XGBRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    random_state=42,
    n_jobs=-1
)

baseline_model.fit(X_train, y_train)

# Evaluate baseline
y_pred_train = baseline_model.predict(X_train)
y_pred_test = baseline_model.predict(X_test)

print("\n--- Baseline Performance ---")
print(f"Train MAE: {mean_absolute_error(y_train, y_pred_train):,.0f} VNĐ")
print(f"Test MAE: {mean_absolute_error(y_test, y_pred_test):,.0f} VNĐ")
print(f"Train RMSE: {np.sqrt(mean_squared_error(y_train, y_pred_train)):,.0f} VNĐ")
print(f"Test RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_test)):,.0f} VNĐ")
print(f"Train R²: {r2_score(y_train, y_pred_train):.4f}")
print(f"Test R²: {r2_score(y_test, y_pred_test):.4f}")

In [ ]:
# Hyperparameter tuning với GridSearchCV
print("\n=== Hyperparameter Tuning ===")
print("Đang tìm kiếm hyperparameters tốt nhất...\n")

param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}

xgb_model = xgb.XGBRegressor(random_state=42, n_jobs=-1)

grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    cv=5,
    scoring='neg_mean_absolute_error',
    verbose=2,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

print("\n--- Best Hyperparameters ---")
print(grid_search.best_params_)

# Best model
best_model = grid_search.best_estimator_

## 8. Đánh giá Model

In [ ]:
# Predictions
y_pred_train_best = best_model.predict(X_train)
y_pred_test_best = best_model.predict(X_test)

# Metrics
print("=== Final Model Performance ===")
print("\n--- Training Set ---")
train_mae = mean_absolute_error(y_train, y_pred_train_best)
train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train_best))
train_r2 = r2_score(y_train, y_pred_train_best)
print(f"MAE: {train_mae:,.0f} VNĐ")
print(f"RMSE: {train_rmse:,.0f} VNĐ")
print(f"R²: {train_r2:.4f}")
print(f"MAPE: {np.mean(np.abs((y_train - y_pred_train_best) / y_train)) * 100:.2f}%")

print("\n--- Test Set ---")
test_mae = mean_absolute_error(y_test, y_pred_test_best)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test_best))
test_r2 = r2_score(y_test, y_pred_test_best)
print(f"MAE: {test_mae:,.0f} VNĐ")
print(f"RMSE: {test_rmse:,.0f} VNĐ")
print(f"R²: {test_r2:.4f}")
print(f"MAPE: {np.mean(np.abs((y_test - y_pred_test_best) / y_test)) * 100:.2f}%")

# Cross-validation
print("\n--- Cross-Validation (5-fold) ---")
cv_scores = cross_val_score(best_model, X_train, y_train, 
                             cv=5, scoring='neg_mean_absolute_error')
print(f"CV MAE: {-cv_scores.mean():,.0f} ± {cv_scores.std():,.0f} VNĐ")

In [ ]:
# Visualization: Actual vs Predicted
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Test set
axes[0].scatter(y_test, y_pred_test_best, alpha=0.5)
axes[0].plot([y_test.min(), y_test.max()], 
             [y_test.min(), y_test.max()], 
             'r--', lw=2)
axes[0].set_xlabel('Giá thực tế (VNĐ)')
axes[0].set_ylabel('Giá dự đoán (VNĐ)')
axes[0].set_title(f'Test Set - R² = {test_r2:.4f}')

# Residuals
residuals = y_test - y_pred_test_best
axes[1].scatter(y_pred_test_best, residuals, alpha=0.5)
axes[1].axhline(y=0, color='r', linestyle='--', lw=2)
axes[1].set_xlabel('Giá dự đoán (VNĐ)')
axes[1].set_ylabel('Residuals (VNĐ)')
axes[1].set_title('Residual Plot')

plt.tight_layout()
plt.show()

In [ ]:
# Feature Importance
feature_importance = pd.DataFrame({
    'feature': feature_columns,
    'importance': best_model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 8))
plt.barh(feature_importance['feature'][:15], 
         feature_importance['importance'][:15])
plt.xlabel('Importance')
plt.title('Top 15 Most Important Features')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print("\n=== Top 10 Important Features ===")
print(feature_importance.head(10))

## 9. Lưu Model

In [ ]:
# Lưu model và metadata
model_data = {
    'model': best_model,
    'feature_columns': feature_columns,
    'metrics': {
        'test_mae': test_mae,
        'test_rmse': test_rmse,
        'test_r2': test_r2
    },
    'feature_importance': feature_importance.to_dict('records')
}

joblib.dump(model_data, 'price_model.pkl')
print("✓ Đã lưu model vào price_model.pkl")

# Download về máy
files.download('price_model.pkl')
print("\n✓ Đã tải model về máy!")
print("\nĐể sử dụng model:")
print("1. Upload file price_model.pkl vào thư mục ml-moderation/models/")
print("2. Model sẽ tự động được load bởi ml_predictor.py")

## 10. Test với dữ liệu mẫu

In [ ]:
# Test với một property mẫu
sample_property = {
    'province_encoded': 1,  # Hồ Chí Minh
    'district_encoded': 5,
    'area': 25,
    'bedrooms': 1,
    'bathrooms': 1,
    'floor': 2,
    'has_wifi': 1,
    'has_parking': 1,
    'has_air_conditioner': 1,
    'has_water_heater': 1,
    'has_kitchen': 1,
    'has_fridge': 1,
    'has_washing_machine': 0,
    'has_tv': 1,
    'has_bed': 1,
    'has_wardrobe': 1,
    'total_amenities': 9,
    'amenity_score': 0.9,
    'area_rooms': 25,
    'area_amenities': 22.5
}

# Convert to DataFrame
sample_df = pd.DataFrame([sample_property])

# Predict
predicted_price = best_model.predict(sample_df)[0]

print("=== Test với Property Mẫu ===")
print("\nThông tin:")
print(f"- Địa điểm: Hồ Chí Minh (province_encoded=1)")
print(f"- Diện tích: {sample_property['area']} m²")
print(f"- Phòng ngủ: {sample_property['bedrooms']}")
print(f"- Phòng tắm: {sample_property['bathrooms']}")
print(f"- Số tiện nghi: {sample_property['total_amenities']}/10")
print(f"\n✓ Giá dự đoán: {predicted_price:,.0f} VNĐ/tháng")

# So sánh với giá trung bình
avg_price = y.mean()
diff_percent = ((predicted_price - avg_price) / avg_price) * 100
print(f"\nSo với giá TB ({avg_price:,.0f} VNĐ): ", end="")
if diff_percent > 0:
    print(f"+{diff_percent:.1f}% (cao hơn)")
else:
    print(f"{diff_percent:.1f}% (thấp hơn)")

## 11. Kết luận

Model XGBoost đã được train thành công!

**Các bước tiếp theo:**
1. ✓ Train XGBoost model để dự đoán giá
2. → Tiếp tục với notebook 3 để train Isolation Forest (anomaly detection)
3. → Upload cả 2 models vào `ml-moderation/models/`
4. → Chạy Flask API và test integration

**Performance Summary:**
- Model có thể dự đoán giá với sai số trung bình khoảng X% (xem MAE/MAPE)
- Features quan trọng nhất: area, location, amenities
- Có thể cải thiện bằng cách thu thập thêm dữ liệu hoặc feature engineering